A complete implementation of a machine learning pipeline using PySpark's MLlib on the Titanic dataset. 
This PROJECT covers:
- Data loading, 
- Preprocessing, 
- Feature engineering, 
- Model training, and 
- Evaluation.

In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, Imputer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

In [2]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("TitanicMLlibExample") \
    .getOrCreate()

In [2]:
x = int(5)
print(type(x))

<class 'int'>


In [6]:
x = "Hello"
y = 'Hello'
if x == y:
    print(True)
else:
    False

True


In [10]:
y = "hello"
x = y.upper()
x

'HELLO'

In [12]:
dic = {
    'x':2,
    'u':5,
    'x':2
}
print(dic)

{'x': 2, 'u': 5}


In [4]:
import seaborn as sns

In [ ]:
# Load the Titanic dataset
titanic = sns.load_dataset('titanic')
titanic.to_csv('titanic.csv')
df = spark.read.csv('../data/titanic.csv', header=True, inferSchema=True)

In [9]:
# Show the schema and first few rows
df.printSchema()
df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)
 |-- class: string (nullable = true)
 |-- who: string (nullable = true)
 |-- adult_male: boolean (nullable = true)
 |-- deck: string (nullable = true)
 |-- embark_town: string (nullable = true)
 |-- alive: string (nullable = true)
 |-- alone: boolean (nullable = true)

+---+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|_c0|survived|pclass|   sex| age|sibsp|parch|   fare|embarked|class|  who|adult_male|deck|embark_town|alive|alone|
+---+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|  0|       0|     3|  male|22.0|    1|   

In [33]:
import requests

# Define the URL and local file name
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
file_path = "titanic.csv"

# Download the file
response = requests.get(url)
with open(file_path, "wb") as f:
    f.write(response.content)

# Now read it with PySpark
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows
df.show(5)


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [36]:
# Data Preprocessing
# 1. Handle missing values
imputer = Imputer(inputCols=["Age", "Fare"], outputCols=["Age_imputed", "Fare_imputed"], strategy="mean")
df = imputer.fit(df).transform(df)

In [35]:
df.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string'),
 ('Age_imputed', 'double'),
 ('Fare_imputed', 'double')]

In [14]:
# 2. Encode categorical variables (e.g., Sex, Embarked)
sex_indexer = StringIndexer(inputCol="Sex", outputCol="Sex_indexed")
embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_indexed")


In [15]:
# 3. Create a feature vector
feature_columns = ["Pclass", "Sex_indexed", "Age_imputed", "SibSp", "Parch", "Fare_imputed", "Embarked_indexed"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")


VectorAssembler_92edc17b5dc5


In [ ]:
# 4. Encode the target variable (Survived)
label_indexer = StringIndexer(inputCol="Survived", outputCol="label")

StringIndexer_b53cd3abb288


In [ ]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()
# missing_values = df.select([sum(col(column).isNull().cast("int")).alias(column) for column in df.columns])
# missing_values.show()


+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+------------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Age_imputed|Fare_imputed|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+------------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|          0|           0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+------------+



In [ ]:
df.assembled = assembler.transform(df)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_imputed', 'Fare_imputed']


In [17]:
# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)


In [18]:
# Build the machine learning pipeline
pipeline = Pipeline(stages=[
    sex_indexer,
    embarked_indexer,
    assembler,
    label_indexer,
    LogisticRegression(featuresCol="features", labelCol="label")
])

In [19]:
# Train the model
model = pipeline.fit(train_data)


Py4JJavaError: An error occurred while calling o123.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 20) (SoG executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda$3833/637340335`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageC

In [ ]:
# Make predictions on the test set
predictions = model.transform(test_data)

In [ ]:
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"AUC: {auc}")

In [ ]:
# Show predictions
predictions.select("PassengerId", "prediction", "label").show(10)



In [37]:
# Stop the Spark session
spark.stop()